<a href="https://colab.research.google.com/github/chihway/cosmology_on_beach_2022/blob/main/Tutorial2_measure_2pt_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Measure 2pt Functions**

Time: 20 min

In this tutorial we will learn to measure 2pt functions from a real shear catalog. 

We will learn to:
* Measure real-space two-point shear correlation functions
* What the different analysis choices does to the measurements

To perform a fast two-point measurement, we rely on a popular package called TreeCorr, from Mike Jarvis (scientist at UPenn). You can read about TreeCorr [here](https://rmjarvis.github.io/TreeCorr/_build/html/index.html). This package is used in papers like [The DES Y3 3x2pt Cosmology](https://arxiv.org/abs/2105.13549).


In [ ]:
import numpy as np
import pylab as mplot
%pylab inline
import astropy.io.fits as pf

In [ ]:
# you only need to install this once
!pip install treecorr

In [ ]:
import treecorr

We will measure the 2pt shear correlation function in the DES Y1 data. You'll need to download the data first. This is a trimmed down version of the im3shape catalog described in [this paper](https://arxiv.org/abs/1708.01533) and results presented in [this paper](https://arxiv.org/abs/1708.01538).

In [ ]:
!rm -rf data_2
!curl -O https://portal.nersc.gov/cfs/lsst/chihway/data_2.tar.gz
!tar -xvzf data_2.tar.gz

Let's read the relevant columns in the file: the positions and shear. Also let's do the measurement on a small patch to speed things up.

In [ ]:
pf.open('data_2/y1a1_im3shape_ellip_and_id.fits.gz')[1].header

In [ ]:
im3shape = pf.open('data_2/y1a1_im3shape_ellip_and_id.fits.gz')[1].data
mask = (im3shape['m']!=0)*(im3shape['ra']>0)*(im3shape['ra']<20)
ra = im3shape['ra'][mask]
dec = im3shape['dec'][mask]
ww = im3shape['weight'][mask]

# shears are "calibrated"
g1 = im3shape['e1'][mask]/np.average(1+im3shape['m'][mask], weights=ww) + np.average(im3shape['c1'][mask], weights=ww)
g2 = im3shape['e2'][mask]/np.average(1+im3shape['m'][mask], weights=ww) + np.average(im3shape['c2'][mask], weights=ww)

g1 -= np.mean(g1)
g2 -= np.mean(g2)

Now a simple treecorr run is just a few lines. This will take a few min.

In [ ]:
cat = treecorr.Catalog(ra=ra, dec=dec, ra_units='degrees', dec_units='degrees', g1=g1, g2=-g2, w=ww, npatch=20)
gg = treecorr.GGCorrelation(min_sep=2., max_sep=250., nbins=20, bin_slop=0.2, sep_units='arcmin', var_method='jackknife')
gg.process(cat)
xip = gg.xip  # The xi_plus correlation function
xim = gg.xim  # The xi_minus correlation function
theta = np.exp(gg.meanlogr)

Let's check out the covariance matrix. First block is xi+, second is xi-.

In [ ]:
cov_jk = gg.estimate_cov('jackknife')
mplot.imshow(cov_jk)

The correlation function is when you normalize the diagonal to 1.

In [ ]:
norm_cov = cov_jk.copy()
for i in range(len(cov_jk)):
  for j in range(len(cov_jk)):
    norm_cov[i][j] /= np.sqrt(cov_jk[i][i]*cov_jk[j][j])
mplot.imshow(norm_cov)

Now plot the results.

In [ ]:
mplot.errorbar(theta, xip, np.diag(cov_jk[:20])**0.5, label='$\\xi_{+}$', color='teal')
mplot.errorbar(theta, xim, np.diag(cov_jk[20:])**0.5, label='$\\xi_{-}$', color='orangered')

mplot.xscale('log')
mplot.xlabel('theta (arcmin)')
mplot.ylabel('$\\xi_{\pm}$')
mplot.legend()


We detected cosmic shear!! \O/

In [ ]:
mplot.errorbar(theta, xip, np.diag(cov_jk[:20])**0.5, label='$\\xi_{+}$', color='teal')
mplot.errorbar(theta, xim, np.diag(cov_jk[20:])**0.5, label='$\\xi_{-}$', color='orangered')

mplot.xscale('log')
mplot.yscale('log')
mplot.xlabel('theta (arcmin)')
mplot.ylabel('$\\xi_{\pm}$')
mplot.legend()
mplot.ylim(1e-8, 1e-4)

Compare your results with your neighbor, are they the same? any guesses why?

### **Try exploring some options**
* changing bin size
* setting this parameter called [`bin_slop`](https://rmjarvis.github.io/TreeCorr/_build/html/binning.html?highlight=slop#bin-slop)
* measuring the 2pt function in different areas in the footprint
* measuring a [3pt function](https://rmjarvis.github.io/TreeCorr/_build/html/correlation3.html)?
* you can also download [this file](http://desdr-server.ncsa.illinois.edu/despublic/y1a1_files/chains/2pt_NG_mcal_1110.fits) to plot directly the final data vector in DES Y1